Following [this](https://pytorch-lightning.readthedocs.io/en/stable/common/early_stopping.html)

### \[Manual\] Stopping an Epoch Early

You can stop and skip the rest of the current epoch early by overriding `on_train_batch_start()` to return `-1`
when some condition is met.

If you do this repeatedly, for every epoch you had originally requested, then this will stop your entire training.

```python
class LitModel(LightningModule):
    def on_train_batch_start():
        if something_something:
            return -1  # Skip!
```

### EarlyStopping Callback

The `EarlyStopping` callback can be used to *monitor a metric* and stop the training when no improvement is observed.

To enable it:
* Import `EarlyStopping` callback.
* Log the metric you want to monitor using `log()` method.
* Init the callback, and set monitor to the logged metric of your choice.
* Set the `mode` based on the metric needs to be monitored.
* Pass the `EarlyStopping` callback to the `Trainer` callbacks flag.

```python
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


class LitModel(LightningModule):
    def validation_step(self, batch, batch_idx):
        loss = ...
        self.log("val_loss", loss)  # NOTE.


model = LitModel()
trainer = Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min")])  # NOTE.
trainer.fit(model)
```

You can customize the callbacks behaviour by changing its parameters.
```python
early_stop_callback = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.00,
    patience=3,
    verbose=False,
    mode="max"
)
trainer = Trainer(callbacks=[early_stop_callback])
```

In case you need early stopping in a different part of training, subclass `EarlyStopping` and change where it is called:

```python
class MyEarlyStopping(EarlyStopping):  # NOTE.
    def on_validation_end(self, trainer, pl_module):
        # override this to disable early stopping at the end of val loop
        pass

    def on_train_end(self, trainer, pl_module):
        # instead, do it at the end of training loop
        self._run_early_stopping_check(trainer)  # NOTE: Like so.
```

**NOTE:**

The `EarlyStopping` callback runs at the *end of every validation epoch* by default.

However, the frequency of validation can be modified by setting various parameters in the `Trainer`, 
for example `check_val_every_n_epoch` and `val_check_interval`.

⚠️ It must be noted that the **patience parameter counts the number of validation checks with no improvement**,
and **not the number of training epochs**.

Therefore, with parameters `check_val_every_n_epoch=10` and `patience=3`,
the trainer will perform at least 40 training epochs before being stopped.